<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/AlmostFinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
################ imports #########################################################################
import ipywidgets as widgets
from IPython.display import display,HTML
import random
from firebase import firebase
import time
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from IPython.display import clear_output
from IPython.display import display
import io
import IPython.display
from matplotlib.figure import Figure
from IPython.display import clear_output

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

global player_details
global USERNAME
global NICKNAME
global LEVEL

USERNAME = 0
NICKNAME = 1
LEVEL = 2

player_details = [0,0,0]

#################################### pie chart creator function ####################################################

def create_pie_chart(right_amount, wrong_amount):
    # Calculate the percentages
    total = right_amount + wrong_amount
    right_percentage = (right_amount / total) * 100
    wrong_percentage = (wrong_amount / total) * 100

    # Create the pie chart
    labels = ['Right', 'Wrong']
    sizes = [right_percentage, wrong_percentage]
    colors = ['blue', 'red']

    fig, ax = plt.subplots(figsize=(4.6, 4.6))  # Increase the graph size
    ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

    # Add titles
    ax.set_title('Your Total Success Percentages', fontsize=16, fontweight='bold')
    ax.legend(labels, title='Results', loc='upper right', fontsize=10)

    chart_widget = widgets.Output()
    with chart_widget:
        plt.show()

    return chart_widget

############################### get function #######################################################

# Function to retrieve all questions from Firebase
def get_questions_by_topic(topic):
    result = FBconn.get('/' + topic, None)
    questions = []
    if result:
        for key, value in result.items():
            options = [value['Option1'], value['Option2'], value['Option3'], value['Option4']]
            question = {
                'key': key,
                'question': value['Question'],
                'options': options,
                'correctOption': value['Correct']
            }
            questions.append(question)
    return questions

####################################### statistic screen ######################################################

def statisticScreen(right_answers, response_times, total_time):

    global player_details
    global USERNAME

    users = FBconn.get('/Users', None)

    if users:
        for user_id, user_data in users.items():
            if user_data['username'] == player_details[USERNAME]:
                right_amount = user_data['rightAnswers']
                wrong_amount = user_data['wrongAnswers']

    # Calculate the average response time for right answers
    right_response_times = [t for t in response_times if t >= 0]
    avg_response_time = np.mean(right_response_times)

    # Generate the data for the bar chart
    categories = list(range(1, len(response_times) + 1))
    response_times = [t if t >= 0 else np.nan for t in response_times]

    # Create the figure and axes for the graph
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(categories, response_times, color='lightblue', alpha=0.8)

    # Set axes labels and title
    ax.set_xlabel('Question')
    ax.set_ylabel('Response Time (seconds)')
    ax.set_title('Quiz Game Results')

    # Customize the graph appearance
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

    # Save the current figure to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Create an IPython Image from this object
    plot_img = IPython.display.Image(buf.read(), format='png')
    buf.close()

    # Create a widgets.Image, ready to be displayed
    plot_widget = widgets.Image(value=plot_img.data, format='png')

    # Close the figure
    plt.close(fig)

    # Create the text annotations for details
    points_text = f'Correct Answers: {right_answers}'
    wrong_answers_text = f'Wrong Answers: {10-right_answers}'
    total_time_text = f'Total Time: {total_time} seconds'
    if right_response_times != []:
        fastest_time_text = f'Fastest Time: {min(right_response_times)} seconds'
    else:
        fastest_time_text = f'Fastest time: None'
    avg_time_text = f'Average Time: {avg_response_time:.2f} seconds'
    details_widget = widgets.HTML(f"<div style='display: inline-block; border-style: dotted; border-color: RoyalBlue; background-color: lightblue; color: black; margin: 15px; padding:25px; font-size:24px;'> {points_text}&nbsp;&nbsp;&nbsp;{wrong_answers_text}&nbsp;&nbsp;&nbsp;{total_time_text}<br>{fastest_time_text}&nbsp;&nbsp;&nbsp;{avg_time_text} </div>")
    details_widget.add_class("details")

    # Create the containers for the graphs and details
    graphs_container = widgets.HBox([plot_widget, create_pie_chart(right_amount, wrong_amount)], layout=widgets.Layout(justify_content='center', align_items='center'))
    graph_details_container = widgets.VBox([graphs_container, details_widget], layout=widgets.Layout(height='auto', justify_content='center', align_items='center'))

    # Close the figure
    plt.close(fig)

    # Create the Back and Try Again buttons
    btn_back = widgets.Button(description='Back', layout=widgets.Layout(width='250px', height='auto'))
    btn_back.add_class("button")
    btn_try_again = widgets.Button(description='Try Again', layout=widgets.Layout(width='250px', height='auto'))
    btn_try_again.add_class("button")

    # Function to handle button clicks
    def handle_button_click(b):
        if b.description == 'Back':
            clear_output(wait=True)
            display(createPlayerScreen())
        elif b.description == 'Try Again':
            clear_output(wait=True)
            display(createSelectionScreen())

    # Assign the button click handlers
    btn_back.on_click(handle_button_click)
    btn_try_again.on_click(handle_button_click)

    # Create a container for the buttons
    buttons_container = widgets.HBox([btn_back, btn_try_again])

    # main_container.children = [container,buttons_container]
    main_container = widgets.VBox([graph_details_container, buttons_container], layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center'))
    main_container.add_class("background-image")

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #4169E1;
        text-align: center;
        font-size: 30px;
        padding: 50px 50px;
        text-transform: uppercase;
      }

      .details {
        text-align: center;
        margin-top: 15px;
      }

      .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 25px 50px 75px 100px;
      }

      .button:hover {
        background-color: #F0F8FF;
        color: #6495ED;
        margin: 25px 50px 75px 100px;
      }

      .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
        margin: 25px 50px 75px 100px;
        transform: translateY(4px);
      }

      .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
      }

      .graph-details-container {
        background-color: rgba(255, 255, 255, 0.8);
        padding: 25px;
      }

      .graphs-container {
        display: flex;
        justify-content: center;
        align-items: center;
        margin-bottom: 20px;
      }
    </style>
    """

    # Return the main_container
    return widgets.VBox([widgets.HTML(css), main_container])

####################################### quiz screen ######################################################

def quizScreen(topic):
    # Create buttons
    back_button = widgets.Button(description="Exit", layout=widgets.Layout(width='100px', height='auto', align_self='flex-start'))
    next_button = widgets.Button(description='Next', layout=widgets.Layout(width='auto', height='auto'))
    next_button.add_class("button")
    back_button.add_class("exit_quiz_button")

    answer_chosen = False

    # Create a question widget
    question_widget = widgets.HTML()
    question_widget.add_class("question")

    # Create a box to hold the 4 options buttons
    options_container = widgets.VBox()

    # Create a box to hold the question and options
    question_container = widgets.VBox([question_widget, options_container ], layout=widgets.Layout(justify_content='center', align_items='center'))

    # Create a list to store burndown icons
    burndown_icons = []

    # Create a burndown graph widget
    burndown_graph = widgets.HTML()
    burndown_graph.layout.margin = '10px'

    # Get all questions
    questions = get_questions_by_topic(topic)

    # Shuffle the questions randomly
    random.shuffle(questions)

    picked = 10 if len(questions) >= 10 else len(questions)

    temp = []

    for i in range(picked):
      temp.append(questions[i])

    questions = temp

    # Set the initial question index to 0
    current_question_index = 0

    # Create an array to store the time taken for each question
    time_array = [-1] * len(questions)

    # Create a variable to store the total time taken
    total_time = 0

    # Function to update the question and options
    def update_question():
        nonlocal answer_chosen
        nonlocal current_question_index
        nonlocal total_time
        answer_chosen = False
        
        # Start the timer for the current question
        start_time = time.time()

        global player_details
        global USERNAME

        # Check if all questions have been answered
        if current_question_index >= len(questions):
            time.sleep(1)
            num_correct = 0
            num_wrong = 0

            for i in range(len(burndown_icons)):
                total_time += time_array[i]
                if burndown_icons[i] == 'X':
                    time_array[i] = -1
                    num_wrong += 1
                else:
                    num_correct += 1
            total_time = round(total_time, 2)
            screen_container.close()
            users = FBconn.get('/Users', None)

            if users:
              for user_id, user_data in users.items():
                  if user_data['username'] == player_details[USERNAME]:
                      my_user = user_data

                      # Update the field values
                      my_user['rightAnswers'] += num_correct
                      my_user['wrongAnswers'] += num_wrong
                      my_user['points'] += num_correct * 10

                      # Add the game statistics object to the "session" array
                      if 'sessions' not in my_user:
                          my_user['sessions'] = []

                      # Create the game statistics object
                      game_stats = {
                          'game_number': (len(my_user['sessions'])+1),
                          'right': num_correct,
                          'wrong': num_wrong,
                          'points': num_correct * 10
                      }

                      my_user['sessions'].append(game_stats)

                      # Update the data in the Firebase Realtime Database
                      FBconn.put('/Users', user_id, my_user)
                      break

            display(statisticScreen(num_correct, time_array, total_time))
            return

        # Get the current question and options
        question = questions[current_question_index]['question']
        options = questions[current_question_index]['options']

        # Update the question widget and options widget
        question_widget.value = f"<p>{question}<p>"
        # Create 4 button widgets for options
        option1_button = widgets.Button(description=options[0])
        option2_button = widgets.Button(description=options[1])
        option3_button = widgets.Button(description=options[2])
        option4_button = widgets.Button(description=options[3])
        options_container.children = [option1_button, option2_button, option3_button, option4_button]
        option1_button.add_class("option-button")
        option2_button.add_class("option-button")
        option3_button.add_class("option-button")
        option4_button.add_class("option-button")
        

        # Function to enable or disable the option buttons and next button
        def buttons_state(state):
          option1_button.disabled = state
          option2_button.disabled = state
          option3_button.disabled = state
          option4_button.disabled = state   
          next_button.disabled = not state
          if state:
              option1_button.add_class("disabled-cursor")
              option2_button.add_class("disabled-cursor")
              option3_button.add_class("disabled-cursor")
              option4_button.add_class("disabled-cursor")
              next_button.remove_class("disabled-cursor")
          else:
              option1_button.remove_class("disabled-cursor")
              option2_button.remove_class("disabled-cursor")
              option3_button.remove_class("disabled-cursor")
              option4_button.remove_class("disabled-cursor")   
              next_button.add_class("disabled-cursor")
       
              
        buttons_state(False)  

        # Define a common function for button clicks
        def option_button_clicked(button):
          nonlocal answer_chosen
          nonlocal burndown_icons
          update_time()
          # Check if an answer is already chosen
          if answer_chosen:
                return

          buttons_state(True)
          # Mark that an answer is chosen
          answer_chosen = True      
          # Check if the selected option is correct
          selected_option = button.description
          correct_option = questions[current_question_index]['correctOption']
          is_correct = selected_option == correct_option

          # Change button colors based on correctness
          if is_correct:
            button.style.button_color = 'lightgreen'
            # Add 'V'  to burndown_icons
            burndown_icons.append('V')
          else:
            button.style.button_color = 'red'
            correct_button = get_correct_option_button(correct_option)
            correct_button.style.button_color = 'lightgreen'
            # Add 'X'  to burndown_icons
            burndown_icons.append('X')

            
        # Function to get the button corresponding to the correct option
        def get_correct_option_button(correct_option):
          if correct_option == options[0]:
             return option1_button
          elif correct_option == options[1]:
            return option2_button
          elif correct_option == options[2]:
            return option3_button
          else: 
            return option4_button

        # Assign the common function to the on_click event of each button
        option1_button.on_click(option_button_clicked)
        option2_button.on_click(option_button_clicked)
        option3_button.on_click(option_button_clicked)
        option4_button.on_click(option_button_clicked)

        
        # Function to calculate and update the time taken for the current question
        def update_time():
            nonlocal current_question_index

            if current_question_index < len(questions):
                current_time = time.time() - start_time
                time_array[current_question_index] = round(current_time, 2)       

    # Function to handle button click event
    def next_button_clicked(button):
        nonlocal current_question_index
        
        # Increment the current_question_index
        if current_question_index<len(questions):
          current_question_index += 1

        # Update the burndown graph
        update_burndown_graph()

        # Update the question and options
        update_question()

    # Function to update the burndown graph
    def update_burndown_graph():
        graph_html = ''
        for icon in burndown_icons:
            if icon=='V':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/rightAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
            elif icon == 'X':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/wrongAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
        graph_html += ''
        burndown_graph.value = graph_html
    def back_button_clicked(button):
        screen_container.close()
        display(createSelectionScreen())

    back_button.on_click(back_button_clicked)
    # Update the question and options initially
    update_question()

    # Create a box to hold the burndown graph, next and back button
    # buttons_container = widgets.HBox([back_button, next_button], layout=widgets.Layout(justify_content='space-between' , width = '800px'))
    buttons_container = widgets.HBox([next_button], layout=widgets.Layout(justify_content='space-between' , width = '800px'))
    # buttons_container2 = widgets.HBox([back_button], layout=widgets.Layout(justify_content='space-between' , width = '800px'))

    # Create a box to hold the entire screen
    screen_container = widgets.VBox([back_button, burndown_graph, question_container, buttons_container],layout=widgets.Layout(width='100%', height='700px', justify_content='center', align_items='center'))
    screen_container.add_class("background-image")

    # Display the screen container
    display(screen_container)

    # Register the button click event handler
    next_button.on_click(next_button_clicked)

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #00079C;
        text-align: center;
        font-size: 20px;
        padding: 50px 50px;
        text-transform: uppercase;
      }
      .exit_quiz_button {
            position: absolute;
            top: 15px;
            left: 15px;
            display: inline-block;
            padding: 15px 25px;
            font-size: 20px;
            cursor: pointer;
            width: 200px;
            text-align: center;
            text-decoration: none;
            outline: none;
            color: #F0F8FF;
            background-color: #6495ED;
            border: none;
            border-radius: 15px;
            box-shadow: 0 9px #4682B4;
        }

      .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
      }

      .button:hover {
        background-color: #F0F8FF;
        color: #6495ED; 
        }

      .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
\      }
     .option-button {
        display: inline-block;
        padding: 10px 30px;
        font-size: 18px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 10px;
        box-shadow: 0 5px #4682B4;
        margin: 10px 0;
        width: 700px;
        line-height: 13px;
      }
      .disabled-cursor {
        cursor: not-allowed !important;
      }
    
      .background-image {
          background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
          background-repeat: no-repeat;
          background-position: center;
          background-size: cover;
      }
    </style>
    """
    display(HTML(css))

####################################### welcome screen #################################################### 

def createWelcomeScreen():

    # Create main logo widget
    logo_widget = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/headindaclouds.png?raw=true' align='center' width='184.5px' height='108.5px'>")
    logo_widget.add_class("main-logo")

    # Create main title widget
    title_widget = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/welcomMsg.png?raw=true' align='center'>")
    title_widget.add_class("main-title")


    def handle_login_button_click(b):
        username = username_input.value
        password = password_input.value

        # Search for the user in the "Users" collection
        users = FBconn.get('/Users', None)

        BEGINNER = 1
        MASTER = 2
        PROFICIENT = 3
        global player_details
        global NICKNAME
        if users:
            for user_id, user_data in users.items():
                if user_data['username'] == username and user_data['password'] == password:
                    if user_data['isAdmin']:
                        screen_container.close()
                        player_details[NICKNAME] = user_data['nickName']
                        display(createManagerScreen())
                    else:
                        screen_container.close()
                        if user_data['points'] >= 1000:
                          user_level = PROFICIENT
                        elif user_data['points'] >= 500:
                          user_level = MASTER
                        else:
                          user_level = BEGINNER

                        player_details = [user_data['username'],user_data['nickName'],user_level]

                        display(createPlayerScreen())
                    return

        # If no matching user found, show error message
        error_label.layout.display = 'block'

    # Create username input widget
    username_input = widgets.Text(placeholder="Username", layout=widgets.Layout(width='300px',padding='20px'))
    username_input.add_class("input-field")

    # Create password input widget
    password_input = widgets.Password(placeholder="Password", layout=widgets.Layout(width='300px',padding='20px'))
    password_input.add_class("input-field")

    # Create login button widget
    login_button = widgets.Button(description="Login", layout=widgets.Layout(max_width='180px', min_height='60px'))
    login_button.add_class("button")
    login_button.on_click(handle_login_button_click)

    # Create error label widget
    error_label = widgets.HTML(value="Invalid username or password.", layout={'display': 'none', 'color': 'red'})

    # Create login container widget
    login_container = widgets.VBox([username_input, password_input, login_button, error_label], layout=widgets.Layout(width='360px', justify_content='center', align_items='center'))
    login_container.add_class("login-container")

    # Create screen container widget
    screen_container = widgets.VBox([logo_widget, title_widget, login_container], layout=widgets.Layout(width='100%', height='900px', justify_content='center', align_items='center'))
    screen_container.add_class("background-image")

    # Apply CSS styles for background image and rounded buttons
    css = """
  <style>
  .main-logo {
    position: absolute;
    top: 20px;
  }

  .main-title {
    margin-bottom: 100px;
    margin-top: 100px;
  }

  .button {
    display: inline-block;
    padding: 15px 25px;
    font-size: 24px;
    cursor: pointer;
    text-align: center;
    text-decoration: none;
    outline: none;
    color: #F0F8FF;
    background-color: #6495ED;
    border: none;
    border-radius: 15px;
    box-shadow: 0 9px #4682B4;
    margin: 25px 0;
  }

  .button:hover {background-color: #F0F8FF; color: #6495ED;}

  .button:active {
    background-color: #4169E1;
    color: #F0F8FF;
    box-shadow: 0 5px #B0E0E6;
    transform: translateY(4px);
  }

  .input-field {
    padding: 10px;
    font-size: 28px;
    font-weight: bolder;
    border-radius: 5px;
    margin-bottom: 10px;
    width: 100%;
    box-sizing: border-box;
    height: 20px;  
  }

  .background-image {
      background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
      background-repeat: no-repeat;
      background-position: center;
      background-size: cover;
  }

  .login-container {
    background-color: rgb(174,218,247);
    border: solid, black, 8px;
    border-radius: 10%;
    display: flex;
    flex-direction: column;
    align-items: center;
    margin: 0 auto;
    background-color: transparent; 
  }
  </style>
  """

    display(HTML(css))

    return screen_container

####################################### add quiz screen #######################################################################

def createAddQuiz():
    global overlay
    global player_details
    global USERNAME
    global NICKNAME
    global LEVEL
    
    def handle_exit_button_click_to_manager(b):
        form.close()
        overlay.close()
        display(createManagerScreen())
    
    form = widgets.VBox(layout=widgets.Layout(width='100%', height='auto', justify_content='flex-start', align_items='center'))
    form.add_class("background-image")
    
    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto', align_self='flex-end', flex='1 1 auto'))
    exit_button.add_class("button")
    exit_button.on_click(handle_exit_button_click_to_manager)
    
    question_input = widgets.Text(placeholder="Question")
    option1_input = widgets.Text(placeholder="Option 1")
    option2_input = widgets.Text(placeholder="Option 2")
    option3_input = widgets.Text(placeholder="Option 3")
    option4_input = widgets.Text(placeholder="Option 4")
    correct_input = widgets.Dropdown(options=["Option 1", "Option 2", "Option 3", "Option 4"])
    topic_choice = widgets.Dropdown(options=["Cloud-Architecture-and-Deployment Model", "Cloud-Computing-Fundamentals", "Cloud-Security", "Cloud-Services-and-Technologies"])

    submit_button = widgets.Button(description="Submit", layout=widgets.Layout(width='250px', height='auto'))
    submit_button.add_class("button")
    error_label = widgets.HTML(value="One of the fields is empty.", layout={'display': 'none', 'color': 'red'})
    success_label = widgets.HTML(value="The quiz added successfully!", layout={'display': 'none', 'color': 'green'})

    # Create the add question container
    add_question = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/addQuestion.png?raw=true' align_items='center'>")
    add_question_container = widgets.HBox([add_question], layout=widgets.Layout(justify_content='center', align_items='center', margin='20px'))

    form.children += (exit_button, add_question_container, question_input, option1_input, option2_input, option3_input, option4_input,
                      topic_choice, correct_input, submit_button, error_label, success_label)

    def handle_submit_button_click(b):
        question = question_input.value
        option1 = option1_input.value
        option2 = option2_input.value
        option3 = option3_input.value
        option4 = option4_input.value
        options = [option1, option2, option3, option4]
        correct = int(correct_input.value.split(" ")[1])
        user_choice = '/' + topic_choice.value

        if not question or not option1 or not option2 or not option3 or not option4 or not correct:
            error_label.layout.display = 'block'
            success_label.layout.display = 'none'
        else:
            # Check if the question already exists
            existing_questions = FBconn.get(user_choice, None)

            if existing_questions:
                existing_questions = existing_questions.values()
                if any(q['Question'] == question for q in existing_questions):
                    error_label.value = "The question already exists."
                    error_label.layout.display = 'block'
                    success_label.layout.display = 'none'
                    return

            # Add the quiz to the database
            data = {'Question': question, 'Option1': options[0], 'Option2': options[1], 'Option3': options[2], 'Option4': options[3], 'Correct': options[correct-1]}
            result = FBconn.post(user_choice, data)

            if result:
                error_label.layout.display = 'none'
                success_label.layout.display = 'block'
                question_input.value = ""
                option1_input.value = ""
                option2_input.value = ""
                option3_input.value = ""
                option4_input.value = ""
            else:
                error_label.value = "Failed to add the quiz to the database. Please try again."

    submit_button.on_click(handle_submit_button_click)

    # Apply CSS styles
    css = """
    <style>
    .title {
      color: #4169E1;
      text-align: center;
      font-size: 30px;
      padding: 50px 50px;
      text-transform: uppercase;
    }

    .button {
      display: inline-block;
      padding: 15px 25px;
      font-size: 24px;
      cursor: pointer;
      text-align: center;
      text-decoration: none;
      outline: none;
      color: #F0F8FF;
      background-color: #6495ED;
      border: none;
      border-radius: 15px;
      box-shadow: 0 9px #4682B4;
    }

    .button:hover {
      background-color: #F0F8FF;
      color: #6495ED;
    }

    .button:active {
      background-color: #4169E1;
      color: #F0F8FF;
      box-shadow: 0 5px #B0E0E6;
      transform: translateY(4px);
    }

    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }

    .combobox {
        width: 250px;
        margin-bottom: 10px;
    }

    .custom-combobox {
      position: relative;
      display: inline-block;
      width: 250px;
    }

    .custom-combobox select {
      display: inline-block;
      width: 100%;
      padding: 5px;
      font-size: 16px;
      outline: none;
      border: none;
      border-radius: 4px;
      background-color: #f1f1f1;
    }

    .custom-combobox select:focus {
      background-color: #ffffff;
    }

    .custom-combobox::after {
      content: '\25BC';
      position: absolute;
      top: 0;
      right: 0;
      padding: 5px;
      background-color: #f1f1f1;
      pointer-events: none;
    }

    .custom-combobox .select-selected {
      background-color: DodgerBlue;
      color: white;
    }

    .custom-combobox .select-selected:hover {
      background-color: DodgerBlue;
    }

    .custom-combobox .select-items div {
      padding: 5px;
      cursor: pointer;
      background-color: #f1f1f1;
      color: black;
      border-radius: 4px;
    }

    .custom-combobox .select-items div:hover {
      background-color: DodgerBlue;
      color: white;
    }
    </style>
    """

    display(widgets.HTML(css))
    overlay.close()
    display(form)

########################################## update quiz screen ####################################################################

def UpdateQuizScreen():
    global questions
    topic_choice = widgets.Dropdown(options=["Cloud-Architecture-and-Deployment Model", "Cloud-Computing-Fundamentals", "Cloud-Security", "Cloud-Services-and-Technologies"])
    question_choice = widgets.Dropdown()
    option1_input = widgets.Text(placeholder="Option 1")
    option2_input = widgets.Text(placeholder="Option 2")
    option3_input = widgets.Text(placeholder="Option 3")
    option4_input = widgets.Text(placeholder="Option 4")
    correct_input = widgets.Dropdown(options=["Option 1", "Option 2", "Option 3", "Option 4"])
    
    # Create labels for the text boxes
    topic_label = widgets.HTML("<b>Topic:</b>", layout=widgets.Layout(width='150px'))
    question_label = widgets.HTML("<b>Question:</b>", layout=widgets.Layout(width='150px'))
    option1_label = widgets.HTML("<b>Option 1:</b>", layout=widgets.Layout(width='150px'))
    option2_label = widgets.HTML("<b>Option 2:</b>", layout=widgets.Layout(width='150px'))
    option3_label = widgets.HTML("<b>Option 3:</b>", layout=widgets.Layout(width='150px'))
    option4_label = widgets.HTML("<b>Option 4:</b>", layout=widgets.Layout(width='150px'))
    correct_label = widgets.HTML("<b>Correct answer:</b>", layout=widgets.Layout(width='150px'))


    # Create HBox containers for each section
    topic_hbox = widgets.HBox([topic_label, topic_choice], layout=widgets.Layout(align_items='center'))
    question_hbox = widgets.HBox([question_label, question_choice], layout=widgets.Layout(align_items='center'))
    option1_hbox = widgets.HBox([option1_label, option1_input], layout=widgets.Layout(align_items='center'))
    option2_hbox = widgets.HBox([option2_label, option2_input], layout=widgets.Layout(align_items='center'))
    option3_hbox = widgets.HBox([option3_label, option3_input], layout=widgets.Layout(align_items='center'))
    option4_hbox = widgets.HBox([option4_label, option4_input], layout=widgets.Layout(align_items='center'))
    correct_hbox = widgets.HBox([correct_label, correct_input], layout=widgets.Layout(align_items='center'))

    update_button = widgets.Button(description="Update")
    update_button.add_class("button")
    message_label = widgets.HTML("")
    message_label.add_class("message-label")

    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto'))
    exit_button.add_class("button")
    exit_button.add_class("exit_button")
    def exit_update_handler(b):
          main_container.close()
          display(createManagerScreen())

    exit_button.on_click(exit_update_handler)

    # Create a VBox container to group all the sections
    HBoxes_container = widgets.VBox([topic_hbox, question_hbox, option1_hbox, option2_hbox, option3_hbox, option4_hbox,
                              correct_hbox], layout=widgets.Layout(justify_content='center', align_items='center'))
    update_label_container = widgets.VBox([update_button,message_label], layout=widgets.Layout(justify_content='space-between', align_items='center'))
    main_container = widgets.VBox([HBoxes_container,update_label_container,exit_button], 
                             layout=widgets.Layout(width='100%', height='500px', justify_content='center', align_items='center'))
    main_container.add_class("background-imaged")

    def update_text_boxes(change):
        global questions
        selected_topic = topic_choice.value
        selected_question = change.new
        questions = get_questions_by_topic(selected_topic)
        # Find the question object based on the selected question
        selected_question_obj = next((question for question in questions if question['question'] == selected_question), None)
          
        # Update the text boxes with the question and options
        if selected_question_obj:
            option1_input.value = selected_question_obj['options'][0]
            option2_input.value = selected_question_obj['options'][1]
            option3_input.value = selected_question_obj['options'][2]
            option4_input.value = selected_question_obj['options'][3]
            for i in range(4):
              if selected_question_obj['correctOption'] == selected_question_obj['options'][i]:
                correct_input.value = "Option " + str(i + 1)
                break 
    # Attach the update_text_boxes function to the question dropdown's value attribute
    question_choice.observe(update_text_boxes, 'value')
          
    def update_question_dropdown(change):
        selected_topic = change.new
        questions = get_questions_by_topic(selected_topic)
        
        # Extract the question text from each question object
        question_choices = [question['question'] for question in questions]
        
        # Update the options of the question dropdown
        question_choice.options = question_choices

    
    # Attach the update_question_dropdown function to the topic dropdown's value attribute
    topic_choice.observe(update_question_dropdown, 'value')
    def update_question(change):
      selected_question = question_choice.value  
      options = [option1_input.value,option2_input.value,option3_input.value,option4_input.value]
      selected_question = question_choice.value
      message_label.value = ""
      if selected_question is None:
          message_label.value = "No question chosen."
          message_label.add_class("message-label-error")

          return
      elif any(option == '' for option in options):
          message_label.value = "One or more options are empty."
          message_label.add_class("message-label-error")

          return
      else:   
        selected_question_obj = next((question for question in questions if question['question'] == selected_question), None)
        if selected_question_obj:
            topic = topic_choice.value
            question_key = selected_question_obj['key']
            question_data = {
                'Question': selected_question,
                'Option1': options[0],
                'Option2': options[1],
                'Option3': options[2],
                'Option4': options[3],
                'Correct': options[int(correct_input.value.split(" ")[1])-1]
            }
            FBconn.put('/' + topic, question_key, question_data)
            message_label.value = "Question updated successfully!"
            message_label.add_class("message-label-success")

    display(main_container)

      
    update_button.on_click(update_question)
    css = """
    <style>
        .exit_button {
            position: absolute;
            top: 8px;
            right:2px;
        }
        .button {
            display: inline-block;
            padding: 6px 25px 4px;
            font-size: 24px;
            cursor: pointer;
            text-align: center;
            text-decoration: none;
            outline: none;
            color: #F0F8FF;
            background-color: #6495ED;
            border: none;
            border-radius: 15px;
            box-shadow: 0 9px #4682B4;
            width:200px;
            height:50px;
        }

        .button:hover {
            background-color: #F0F8FF;
            color: #6495ED;
        }

        .button:active {
            background-color: #4169E1;
            color: #F0F8FF;
            box-shadow: 0 5px #B0E0E6;
        }

        .background-imaged {
            background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
            background-repeat: no-repeat;
            background-position: center;
            background-size: cover;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        
        .message-label {
            font-weight: bold;
            padding: 10px;
            border-radius: 15px;
            margin-top: 10px;
            color: #00AA00;
            width:250px;
            text-align: center;
            min-height: 50px;
        }
        
        .message-label-error {
            background-color: #FFCCCC;
            color: #FF0000;
        }
        
        .message-label-success {
            background-color: #CCFFCC;
            color: #00AA00;
        }
    </style>
    """

    display(HTML(css))

####################################### remove quiz screen #######################################################################

def createRemoveQuiz():
    global topic_choice  # Declare topic_choice as a global variable

    def removeQuestion(questionId):
        FBconn.delete(f'{chosen_topic}/{questionId}', None)
        question_element = next((c for c in remove_quiz_container.children if getattr(c, 'question_id', None) == questionId), None)
        if question_element:
            remove_quiz_container.children = tuple(c for c in remove_quiz_container.children if c != question_element)

    def handle_exit_button_click_to_manager(b):
        remove_quiz_container.close()
        exit_button.close()
        overlay.close()
        display(createManagerScreen())

    def handle_topic_change(change):
        chosen_topic = '/' + change.new
        questions = FBconn.get(chosen_topic, None)
        question_containers = []

        if questions:
            for question_id, question_data in questions.items():
                question_container = widgets.HBox(layout=widgets.Layout(width='100%', height='100%', margin='-5px 0px -5px 0px'))
                question_text = widgets.HTML(f"<p style='font-size:16px; color:#000000; text-align=left;margin-left: 80px;font-weight: bolder;'>{question_data['Question']}</p>")
                remove_button = widgets.Button(description="Remove", layout=widgets.Layout(width='100px', height='100%', font_size='1px', padding='0px 5px', border_radius='20px'))
                remove_button.add_class("button")
                remove_button.question_id = question_id
                remove_button.on_click(lambda b, q_id=question_id: removeQuestion(q_id))
                question_container.children = [question_text, remove_button]
                question_container.question_id = question_id
                question_containers.append(question_container)

        remove_quiz_container.children = [exit_button, remove_question_container, topic_choice] + question_containers
        reset_question_appearance()

    def reset_question_appearance():
        for question_container in question_containers:
            question_container.children[0].add_class("question-text")
            question_container.children[1].remove_class("question-button-wet")
            question_container.children[1].remove_class("question-button-dry")
            question_container.children[1].add_class("question-button")

    topic_choice = widgets.Dropdown(options=["Cloud-Architecture-and-Deployment Model", "Cloud-Computing-Fundamentals", "Cloud-Security", "Cloud-Services-and-Technologies"])

    chosen_topic = '/' + topic_choice.value

    remove_quiz_container = widgets.VBox(layout=widgets.Layout(width='100%', height='100%', justify_content='center', align_items='center'))
    remove_quiz_container.add_class("background-image")

    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto',height='auto', align_self='flex-end'))
    exit_button.add_class("button")
    exit_button.on_click(handle_exit_button_click_to_manager)

    # Create the add question container
    remove_question = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/removeQuestion.png?raw=true' align_items='center'>")
    remove_question_container = widgets.HBox([remove_question], layout=widgets.Layout(flex_direction='column', justify_content='center', align_items='center', margin='20px'))

    questions = FBconn.get(chosen_topic, None)
    question_containers = []

    if questions:
        for question_id, question_data in questions.items():
            question_container = widgets.HBox(layout=widgets.Layout(width='100%', height='100%', margin='-5px 0px -5px 0px'))
            question_text = widgets.HTML(f"<p style='font-size:16px; color:#000000;text-align=left;margin-left: 80px;font-weight: bolder;'>{question_data['Question']}</p>")
            remove_button = widgets.Button(description="Remove", layout=widgets.Layout(width='100px', height='100%', font_size='1px', padding='0px 5px', border_radius='20px'))
            remove_button.add_class("button")
            remove_button.question_id = question_id  # Add the question_id attribute
            remove_button.on_click(lambda b, q_id=question_id: removeQuestion(q_id))
            question_container.children = [question_text, remove_button]
            question_container.question_id = question_id
            question_container.add_class("question-container")  # Add the question-container class
            question_containers.append(question_container)

    remove_quiz_container.children = [exit_button, remove_question_container, topic_choice] + question_containers

    topic_choice.observe(handle_topic_change, 'value')

    # Apply CSS styles
    css = """
          <style>
          .main-logo {
            position: absolute;
            top: 20px;
          }

          .main-title {
            margin-bottom: 100px;
            margin-top: 100px;
          }
          
          .button {
            display: inline-block;
            cursor: pointer;
            text-align: center;
            text-decoration: none;
            outline: none;
            color: #F0F8FF;
            font_size: 5px;
            background-color: #6495ED;
            border: none;
            box-shadow: 0 4px #4682B4;
          }

          .button:hover {
            background-color: #F0F8FF;
            color: #6495ED;
          }

          .button:active {
            background-color: #4169E1;
            color: #F0F8FF;
          }

          .input-field {
            display: block;
            padding: 10px;
            font-size: 20px;
            border-radius: 5px;
            border: 1px solid #ccc;
            margin-bottom: 10px;
            width: 100%;
            box-sizing: border-box;
          }

          .background-image {
              background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
              background-repeat: no-repeat;
              background-position: center;
              background-size: cover;
          }

          .login-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            margin: 0 auto;
            max-width: 300px;
          }

          .question-text {
            font-size: 15px;
            color: #bee6ff;
            text-align: left;
            margin-left: 50px;
          }

          .question-button {
            display: inline-block;
            padding: 0px 5px;
            font-size: 1px;
            cursor: pointer;
            text-align: center;
            text-decoration: none;
            outline: none;
            color: #F0F8FF;
            background-color: #6495ED;
            border: none;
            border-radius: 20px;
            box-shadow: 0 4px #4682B4;
            margin-left: 10px;
          }

        .question-container {
            height: 50px;
          }
          </style>
          """

    overlay.close()
    display(widgets.HTML(css))  # Display the CSS styles
    display(remove_quiz_container)

####################################### manager screen #######################################################################

def createManagerScreen():
    global overlay  # Declare overlay as a global variable

    # Create the overlay widget
    overlay = widgets.Output()

    manager_screen = widgets.VBox()

    with overlay:
        def handle_exit_button_click_to_popup(b):
            overlay.close()
            manager_screen.close()
            display(createWelcomeScreen())

        def handle_add_quiz_button_click(b):
            overlay.close()
            manager_screen.close()
            createAddQuiz()

        def handle_remove_quiz_button_click(b):
            overlay.close()
            manager_screen.close()
            createRemoveQuiz()

        def handle_update_quiz_button_click(b):
          overlay.close()
          manager_screen.close()
          UpdateQuizScreen()

        # Create the exit button
        exit_button = widgets.Button(description="X", layout=widgets.Layout(width='100px', height='auto', align_self='flex-end'))
        exit_button.add_class("button")
        exit_container = widgets.HBox([exit_button], layout=widgets.Layout(align_self='flex-end'))
        exit_button.on_click(handle_exit_button_click_to_popup)

        # Create the manager title
        manager_title = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/properties.png?raw=true' align_items='center'>")
        title_container = widgets.HBox([manager_title], layout=widgets.Layout(justify_content='center', align_items='center', margin='50px'))

        global player_details
        global NICKNAME
        # Create the manager title
        manager_title = widgets.HTML("<h1 style='text-align: center; font-size: 40px; color: #003366; padding-top: 50px;'>Welcome " + player_details[NICKNAME] + "!</h1>")

        # Create the add quiz button
        add_quiz_button = widgets.Button(description="Add Quiz", layout=widgets.Layout(width='250px', height='auto', margin='25px auto'))
        add_quiz_button.add_class('add_quiz_button')
        add_quiz_button.add_class("button")
        add_quiz_button.on_click(handle_add_quiz_button_click)

        # Create the remove quiz button
        remove_quiz_button = widgets.Button(description="Remove Quiz", layout=widgets.Layout(width='250px', height='auto', margin='25px auto'))
        remove_quiz_button.add_class('remove_button')
        remove_quiz_button.add_class("button")
        remove_quiz_button.on_click(handle_remove_quiz_button_click)

        # Create the remove quiz button
        update_quiz_button = widgets.Button(description="Update Quiz", layout=widgets.Layout(width='250px', height='auto', margin='25px auto'))
        update_quiz_button.add_class('update_button')
        update_quiz_button.add_class("button")
        update_quiz_button.on_click(handle_update_quiz_button_click)

        # Create a container for the buttons
        buttons_container = widgets.VBox([add_quiz_button, remove_quiz_button, update_quiz_button], layout=widgets.Layout(justify_content='center', align_items='center', margin='25px'))

        # Create the manager screen widget
        manager_screen.children = [exit_container, manager_title, title_container, buttons_container]
        manager_screen.add_class("background-image")

        # Apply CSS styles
        css = """
        <style>
        .main-logo {
          position: absolute;
          top: 20px;
        }

        .main-title {
          margin-bottom: 100px;
          margin-top: 100px;
        }

        .button {
          display: block;
          padding: 15px 25px;
          font-size: 24px;
          cursor: pointer;
          text-align: center;
          text-decoration: none;
          outline: none;
          color: #F0F8FF;
          background-color: #6495ED;
          border: none;
          border-radius: 15px;
          box-shadow: 0 9px #4682B4;
          margin: 25px auto;
          width: 100%;
          max-width: 300px;
        }

        .button:hover {
          background-color: #F0F8FF;
          color: #6495ED;
        }

        .button:active {
          background-color: #4169E1;
          color: #F0F8FF;
          box-shadow: 0 5px #B0E0E6;
          transform: translateY(4px);
        }

        .input-field {
          display: block;
          padding: 10px;
          font-size: 20px;
          border-radius: 5px;
          border: 1px solid #ccc;
          margin-bottom: 10px;
          width: 100%;
          box-sizing: border-box;
        }

        .background-image {
            background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
            background-repeat: no-repeat;
            background-position: center;
            background-size: cover;
        }

        .login-container {
          display: flex;
          flex-direction: column;
          align-items: center;
          justify-content: center;
          margin: 0 auto;
          max-width: 300px;
          width: 100%;
          height: 100%;
        }
        </style>
        """


    # Display the manager screen widget
    display(manager_screen)

    return overlay


######################################### player screen ############################################################333

def createPlayerScreen():

    global player_details
    global USERNAME
    global NICKNAME
    global LEVEL

    # Create the player screen widget
    player_screen = widgets.VBox()

    def handle_start_play_button_click(b):
        player_screen.close()
        display(createSelectionScreen())
        # quizScreen()

    def handle_statistics_button_click(b):
        print("statistics")

    # Create the exit button
    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto'))
    exit_button.add_class("button")
    exit_button.add_class("exit_button")

    player_level = ""

    if player_details[LEVEL] == 1:
        player_level = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/bronze.png?raw=true' style='width: 40px; height: 53px;'>")
    elif player_details[LEVEL] == 2:
        player_level = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/silver.png?raw=true' style='width: 40px; height: 53px;'>")
    else:
        player_level = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/gold.png?raw=true' style='width: 40px; height: 53px;'>")

    nick = widgets.HTML(f"<h1 style='color: #003366; font-size: 30px; margin: 10px;'>{player_details[NICKNAME]}</h1>")

    nick_level_container = widgets.HBox([nick, player_level])

    # Create the player title
    player_title = widgets.HTML(f"<div><img src='https://github.com/EyalKoubi/CloudProject/blob/main/gameInfo.png?raw=true' style='display: block; margin: auto;width: 450px; height: auto;' /></div>")

    # Create the "Start Play" button
    start_play_button = widgets.Button(description="SELECT TOPIC", layout=widgets.Layout(width='300px', height='auto'))
    start_play_button.add_class("button")
    start_play_button.on_click(handle_start_play_button_click)

    # Create screen container widget
    player_screen = widgets.VBox([nick_level_container, player_title, start_play_button, exit_button], layout=widgets.Layout(position='absolute', top='0', left='0'))
    player_screen.add_class("background-image")

    def exit_player_handler(b):
        player_screen.close()
        display(createWelcomeScreen())

    exit_button.on_click(exit_player_handler)

    # Apply CSS styles for rounded buttons
    css = """
    <style>
    .exit_button {
        position: absolute;
        top: 8px;
        right: 2px;
    }
    
    .button {
        display: block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 25px auto;
        max-width: 300px;
    }

    .button:hover {background-color: #F0F8FF; color: #6495ED; }

    .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
    }
    
    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: top left;
        background-size: cover;
    }
    </style>
    """
    display(HTML(css))

    return player_screen

######################################## select topic screen ###################################################################

# Function to create the selection screen
def createSelectionScreen():
    # Define the topics
    topics = ["Cloud-Services-and-Technologies", "Cloud-Security", "Cloud-Architecture-and-Deployment Model", "Cloud-Computing-Fundamentals"]

    # Create the selection screen widget
    selection_screen = widgets.VBox()

    def handle_topic_selection(button):
        selection_screen.close()
        quizScreen(button.description)

    # Create the exit button
    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto'))
    exit_button.add_class("button")

    # Create button container widget
    exit_btn_container = widgets.HBox([exit_button])
    exit_btn_container.add_class("exit_button")

    # Create the selection title
    selection_title = widgets.HTML("<h2>Select a Topic:</h2>")
    selection_title.add_class("selection-title")

    # Create the topic buttons
    topic_buttons = []
    for topic in topics:
        button = widgets.Button(description=topic, layout=widgets.Layout(width='auto', height='auto'))
        button.add_class("topic-button")
        button.on_click(handle_topic_selection)
        topic_buttons.append(button)

    # Create the topics container
    topics_container = widgets.VBox(topic_buttons)

    # Create screen container widget
    selection_screen = widgets.VBox([selection_title, topics_container, exit_btn_container],
                                    layout=widgets.Layout(width='100%', height='600px', justify_content='center',
                                                          align_items='center'))
    selection_screen.add_class("screen-container")
    selection_screen.add_class("background-image")

    def exit_selection_handler(b):
        selection_screen.close()
        display(createPlayerScreen())

    exit_button.on_click(exit_selection_handler)

    # Apply CSS styles for buttons and background
    css = """
    <style>
    .selection-title{
        color: #00079C;
        text-align: center;
        font-size: 20px;
        padding: 50px 50px;
        text-transform: uppercase;
    }

    .exit_button {
        position: absolute;
        top: 8px;
        right:2px;
    }
    
    .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
    }

    .button:hover {
        background-color: #F0F8FF;
        color: #6495ED; 
        }

    .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
    }

    .topic-button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 10px 20px;
    }

    .topic-button:hover {
        background-color: #F0F8FF;
        color: #6495ED;
        margin: 10px 20px;
    }

    .topic-button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
        margin: 10px 20px;
        transform: translateY(4px);
    }

    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }
    </style>
    """
    display(HTML(css))

    return selection_screen

####################################### activate screens #######################################################################

screen_container = createWelcomeScreen()
display(screen_container)


In [44]:
3

3